In [9]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

printjobs = pd.read_json('3dp/2020-09-12.json')
printjobs.describe()
print(printjobs.columns.values.tolist())
print(printjobs['client_msg_id'])
sum(printjobs['parent_user_id'].isnull())


['client_msg_id', 'type', 'text', 'user', 'ts', 'team', 'user_team', 'source_team', 'user_profile', 'blocks', 'thread_ts', 'reply_count', 'reply_users_count', 'latest_reply', 'reply_users', 'replies', 'is_locked', 'subscribed', 'last_read', 'parent_user_id', 'files', 'upload', 'display_as_bot', 'reactions']
0     c7a3ea41-c795-4839-83aa-7d0db9852578
1     b2d68d32-bd2d-4da2-ada4-75824d36dd47
2     5787119a-a986-4cf5-a9f0-f61d547f0df8
3     0eb0e3c2-306c-4841-92c6-6cb09942525d
4     e50ffde9-ed60-4db6-a1b4-7480741ba183
5     46112242-3ea5-42a8-8325-1bfdac97dbeb
6     dc3f418f-88c9-41c7-a380-19cda24559c7
7     d7894020-92a4-4926-955b-3057da7e3dd4
8     ddbe3f42-117b-44a1-9a1f-d6e2edf92948
9     cde1e4f5-41e8-4c63-95b6-f989c7be4cb7
10    b667b57e-856b-45b3-a947-8ce8ece9558f
11                                     NaN
12    145a7ee9-2cee-4b3e-874c-a0a49480955f
13    26BC3145-E7F4-42DC-AE05-C3E103673689
14    1ff2211b-6e5d-4369-ba7b-b05460653578
15    66c718bf-4744-4c80-83a6-bf52db334af8
16 

21